In [ ]:
#My very first hive query!!!

I have a table as follows:

SELECT a.storeid, a.smonth, a.sales FROM table a;
1001    1       35000.0
1002    2       35000.0
1001    2       25000.0
1002    3       110000.0
1001    3       40000.0
1002    1       40000.0
My objective output is as follows:

1001    1       35000.0 35000.0
1001    2       25000.0 60000.0
1001    3       40000.0 100000.0
1002    1       40000.0 40000.0
1002    2       35000.0 75000.0
1002    3       110000.0 185000.0


SELECT storeid, smonth, sales, SUM(sales) OVER (PARTITION BY storeid ORDER BY smonth) FROM table_name;

In [ ]:
#Ought to mention other variations to achieve the same result

Alternatively, you might have a look at this Hive ticket which contains several feature extensions.
Among others there's a cumulative sum implementation (GenericUDFSum).

This function (referred to as "rsum") takes two arguments, the hash of the id (by which the records are partitioned among the reducers) and their corresponding values to be summed up:

select t.storeid, t.smonth, t.sales, rsum(hash(t.storeid),t.sales) as sales_sum 
from (select storeid, smonth, sales from sales_sum distribute by hash(storeid) 
sort by storeid, smonth) t;

1001  1  35000.0  35000.0
1001  2  25000.0  60000.0
1001  3  40000.0  100000.0
1002  1  40000.0  40000.0
1002  2  35000.0  75000.0
1002  3  110000.0 185000.0

In [ ]:
#Not sure whether the below one works but found it too

SELECT 
    a.storeid, 
    a.smonth,
    a.sales,
    SUM(a.sales) 
OVER (
    PARTITION BY a.storeid 
    ORDER BY a.smonth asc 
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
FROM 
    table a;